## Import libraries

In [348]:
import warnings
warnings.filterwarnings("ignore")

In [349]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_recall_fscore_support
from sklearn.metrics import f1_score,roc_auc_score
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from xgboost import plot_importance

## Anomaly-based IDS - Infiltration

### Generate the port-scan datasets for unknown attack detection

In [401]:
df=pd.read_csv('CICIDS2017_sample_km.csv')

In [402]:
df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,2.565000,-0.439152,-0.005817,-0.009418,-0.040156,-0.007139,-0.211382,-0.210703,-0.153699,-0.164849,...,0.002545,-0.125734,-0.104565,-0.149326,-0.101016,-0.351926,-0.109460,-0.356868,-0.338993,0
1,2.759600,-0.439341,-0.009819,-0.009418,-0.051263,-0.007139,-0.246240,-0.210703,-0.213347,-0.182201,...,0.002545,-0.125734,-0.104565,-0.149326,-0.101016,-0.351926,-0.109460,-0.356868,-0.338993,0
2,2.725909,-0.439340,-0.009819,-0.009418,-0.051863,-0.007139,-0.246240,-0.310140,-0.229468,-0.167112,...,0.002556,-0.125734,-0.104565,-0.149326,-0.101016,-0.351926,-0.109460,-0.356868,-0.338993,0
3,2.967710,-0.439345,-0.009819,-0.010421,-0.051263,-0.007142,-0.246240,-0.210703,-0.213347,-0.182201,...,0.002545,-0.125734,-0.104565,-0.149326,-0.101016,-0.351926,-0.109460,-0.356868,-0.338993,0
4,2.683412,-0.430518,-0.008485,-0.008416,-0.045960,-0.007137,-0.180706,-0.210703,-0.151549,-0.097234,...,0.002545,-0.125734,-0.104565,-0.149326,-0.101016,-0.351926,-0.109460,-0.356868,-0.338993,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25485,-0.417175,2.529503,0.022195,0.015647,0.612863,-0.007073,1.333548,-0.210703,1.066630,1.056629,...,0.002545,0.847478,0.135514,0.572316,0.898203,1.023440,4.126922,1.900294,0.555568,9
25486,-0.417175,2.871035,0.015525,0.010634,0.403729,-0.007086,0.693545,-0.210703,0.860244,0.661178,...,0.002545,0.773475,-0.099716,0.420533,0.907276,1.967152,-0.109145,1.892086,2.006554,9
25487,-0.417175,1.922710,0.042204,0.028681,1.412575,-0.007038,1.746272,-0.210703,1.609455,1.434458,...,0.002545,4.152361,7.829669,4.707039,0.882819,1.201179,5.539049,1.904398,0.444283,9
25488,-0.417175,1.000343,-0.003150,-0.002400,0.141060,-0.007121,1.053285,-0.210703,1.191111,1.068873,...,0.002545,-0.125734,-0.104565,-0.149326,-0.101016,-0.351926,-0.109460,-0.356868,-0.338993,9


In [403]:
df.Label.value_counts()

,count
Label,
0,10226
4,4619
10,3178
2,2564
1,1966
12,1507
14,652
3,208
7,155


In [404]:
df1 = df[df['Label'] != 10]
df1['Label'][df1['Label'] > 0] = 1
df1.to_csv('./CICIDS2017_sample_km_without_infiltration.csv',index=0)

In [405]:
df2 = df[df['Label'] == 10]
df2['Label'][df2['Label'] == 10] = 1
df2.to_csv('./CICIDS2017_sample_km_infiltration.csv',index=0)

### Read the generated datasets for unknown attack detection

In [406]:
df1 = pd.read_csv('./CICIDS2017_sample_km_without_infiltration.csv')
df2 = pd.read_csv('./CICIDS2017_sample_km_infiltration.csv')

In [407]:
features = df1.drop(['Label'],axis=1).dtypes[df1.dtypes != 'object'].index
df1[features] = df1[features].apply(
    lambda x: (x - x.mean()) / (x.std()))
df2[features] = df2[features].apply(
    lambda x: (x - x.mean()) / (x.std()))
df1.replace([np.inf, -np.inf], np.nan, inplace=True)
df2.replace([np.inf, -np.inf], np.nan, inplace=True)
df1 = df1.fillna(0)
df2 = df2.fillna(0)

In [408]:
df1.Label.value_counts()

,count
Label,
1,12086
0,10226


In [409]:
df2.Label.value_counts()

,count
Label,
1,3178


In [410]:
df2p=df1[df1['Label']==0]
df2pp=df2p.sample(n=None, frac=3178/10226, replace=False, weights=None, random_state=None, axis=0)
df2=pd.concat([df2, df2pp])

In [411]:
df2.Label.value_counts()

,count
Label,
1,3178
0,3178


In [412]:
df = pd.concat([df1, df2])

In [413]:
df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,3.140998,-0.551435,-0.010833,-0.012378,-0.036492,-0.010315,-0.170008,-0.101966,-0.105396,-0.172888,...,-0.970844,-0.131346,-0.085516,-0.143181,-0.115532,-0.468705,-0.139555,-0.476791,-0.451732,0
1,3.368774,-0.551605,-0.014487,-0.012378,-0.039738,-0.010315,-0.198737,-0.101966,-0.153339,-0.190956,...,-0.970844,-0.131346,-0.085516,-0.143181,-0.115532,-0.468705,-0.139555,-0.476791,-0.451732,0
2,3.329339,-0.551604,-0.014487,-0.012378,-0.039914,-0.010315,-0.198737,-0.210150,-0.166296,-0.175244,...,0.975904,-0.131346,-0.085516,-0.143181,-0.115532,-0.468705,-0.139555,-0.476791,-0.451732,0
3,3.612362,-0.551609,-0.014487,-0.013319,-0.039738,-0.010318,-0.198737,-0.101966,-0.153339,-0.190956,...,-0.970844,-0.131346,-0.085516,-0.143181,-0.115532,-0.468705,-0.139555,-0.476791,-0.451732,0
4,3.279597,-0.543679,-0.013269,-0.011436,-0.038188,-0.010313,-0.144726,-0.101966,-0.103669,-0.102477,...,-0.970844,-0.131346,-0.085516,-0.143181,-0.115532,-0.468705,-0.139555,-0.476791,-0.451732,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9646,0.916501,-0.551608,-0.014487,-0.011436,-0.040703,-0.010313,-0.232062,-0.174088,-0.224605,-0.256423,...,-0.321928,-0.131346,-0.085516,-0.143181,-0.115532,-0.468705,-0.139555,-0.476791,-0.451732,0
1046,-0.372885,2.607653,0.003782,0.000803,-0.012863,-0.009632,0.266675,-0.210150,0.009645,0.272959,...,-0.970844,-0.080523,-0.015673,-0.039045,-0.077138,-0.166306,-0.045243,-0.176948,-0.205769,0
2761,-0.374613,-0.550306,-0.014487,-0.011436,-0.038949,-0.010242,-0.197588,0.366827,-0.095030,-0.256423,...,0.975904,-0.131346,-0.085516,-0.143181,-0.115532,-0.468705,-0.139555,-0.476791,-0.451732,0
7759,-0.374613,-0.551604,-0.014487,-0.011436,-0.035907,-0.010136,-0.137831,1.304415,0.129565,-0.256423,...,-0.970844,-0.131346,-0.085516,-0.143181,-0.115532,-0.468705,-0.139555,-0.476791,-0.451732,0


In [414]:
X = df.drop(['Label'],axis=1) .values
y = df.iloc[:, -1].values.reshape(-1,1)
y=np.ravel(y)
pd.Series(y).value_counts()

,count
1,15264
0,13404


### Feature engineering (IG, FCBF, and KPCA)

#### Feature selection by information gain (IG)

In [415]:
from sklearn.feature_selection import mutual_info_classif
importances = mutual_info_classif(X, y)

In [416]:
# calculate the sum of importance scores
f_list = sorted(zip(map(lambda x: round(x, 4), importances), features), reverse=True)
Sum = 0
fs = []
for i in range(0, len(f_list)):
    Sum = Sum + f_list[i][0]
    fs.append(f_list[i][1])

In [417]:
# select the important features from top to bottom until the accumulated importance reaches 90%
f_list2 = sorted(zip(map(lambda x: round(x, 4), importances/Sum), features), reverse=True)
Sum2 = 0
fs = []
for i in range(0, len(f_list2)):
    Sum2 = Sum2 + f_list2[i][0]
    fs.append(f_list2[i][1])
    if Sum2>=0.9:
        break

In [418]:
X_fs = df[fs].values

In [419]:
X_fs.shape

(28668, 51)

In [420]:
X_fs

array([[ 3.1409979 , -0.55374567, -0.58212483, ..., -0.14318079,
        -0.47679117, -0.45173199],
       [ 3.36877385, -0.505954  , -0.60494727, ..., -0.14318079,
        -0.47679117, -0.45173199],
       [ 3.32933878, -0.49424991, -0.60936451, ..., -0.14318079,
        -0.47679117, -0.45173199],
       ...,
       [-0.37461305, -0.57955467, -0.50924029, ..., -0.14318079,
        -0.47679117, -0.45173199],
       [-0.37461305, -0.57955467, -0.28616942, ..., -0.14318079,
        -0.47679117, -0.45173199],
       [ 3.32383323, -0.42575102, -0.64175764, ..., -0.14318079,
        -0.47679117, -0.45173199]])

#### Feature selection by Fast Correlation Based Filter (FCBF)

The module is imported from the GitHub repo: https://github.com/SantiagoEG/FCBF_module

In [33]:
!pip install scikit-optimizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 519.9 kB/s eta 0:00:00


In [421]:
from skopt import gp_minimize
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

def fcbf_objective(threshold, X, y, clf=RandomForestClassifier(random_state=42)):
    selector = FCBF(th=threshold)
    X_selected = selector.fit_transform(X, y)
    if X_selected.shape[1] == 0:
        return 1.0
    score = cross_val_score(clf, X_selected, y, cv=3, scoring="accuracy").mean()
    return -score

In [422]:
from FCBF_module import FCBF, FCBFK, FCBFiP, get_i
fcbf = FCBF(th = 0.09988304703442026)

In [423]:
X_fss = fcbf.fit_transform(X_fs,y)

In [424]:
X_fss.shape

(28668, 1)

In [425]:
X_fss

array([[ 3.1409979 ],
       [ 3.36877385],
       [ 3.32933878],
       ...,
       [-0.37461305],
       [-0.37461305],
       [ 3.32383323]])

####  kernel principal component analysis (KPCA)

In [ ]:
from skopt import gp_minimize
from skopt.space import Integer, Categorical
from sklearn.decomposition import KernelPCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

def kpca_objective(params, X, y):
    n_components, kernel = params
    n_components = min(n_components, X.shape[1] - 1)
    if n_components < 1: n_components = 1
    try:
        kpca = KernelPCA(n_components=n_components, kernel=kernel, fit_inverse_transform=False, random_state=42)
        X_kpca = kpca.fit_transform(X)
        clf = RandomForestClassifier(random_state=42)
        score = cross_val_score(clf, X_kpca, y, cv=3, scoring="accuracy").mean()
        return -score
    except Exception as e:
        return 1.0

search_space = [
    Integer(10, 50),
    Categorical(['rbf', 'poly'])
]

result = gp_minimize(
    lambda params: kpca_objective(params, X_fss, y),
    search_space,
    n_calls=20,
    random_state=42,
    acq_func='EI'
)

best_n_components, best_kernel = result.x
print(f"Best n_components: {best_n_components}, Best kernel: {best_kernel}")
print(f"Best accuracy: {-result.fun:.4f}")

In [426]:
from skopt import gp_minimize
from skopt.space import Integer, Categorical
from sklearn.decomposition import KernelPCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

kpca = KernelPCA(n_components=20, kernel='rbf', random_state=42)
X_kpca = kpca.fit_transform(X_fss)

In [427]:
X_train = X_kpca[:len(df1)]
y_train = y[:len(df1)]
X_test = X_kpca[len(df1):]
y_test = y[len(df1):]

In [428]:
pd.Series(y_train).value_counts()

,count
1,12086
0,10226


In [429]:
from imblearn.over_sampling import SMOTE
smote=SMOTE(sampling_strategy={0:12086})
X_train, y_train = smote.fit_resample(X_train, y_train)

In [430]:
pd.Series(y_train).value_counts()

,count
0,12086
1,12086


In [431]:
pd.Series(y_test).value_counts()

,count
1,3178
0,3178


### Apply the cluster labeling (CL) k-means method

In [432]:
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN,MeanShift
from sklearn.cluster import SpectralClustering,AgglomerativeClustering,AffinityPropagation,Birch,MiniBatchKMeans,MeanShift
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.metrics import classification_report
from sklearn import metrics

In [433]:
def CL_kmeans(X_train, X_test, y_train, y_test,n,b=100):
    km_cluster = MiniBatchKMeans(n_clusters=n,batch_size=b)
    result = km_cluster.fit_predict(X_train)
    result2 = km_cluster.predict(X_test)

    count=0
    a=np.zeros(n)
    b=np.zeros(n)
    for v in range(0,n):
        for i in range(0,len(y_train)):
            if result[i]==v:
                if y_train[i]==1:
                    a[v]=a[v]+1
                else:
                    b[v]=b[v]+1
    list1=[]
    list2=[]
    for v in range(0,n):
        if a[v]<=b[v]:
            list1.append(v)
        else:
            list2.append(v)
    for v in range(0,len(y_test)):
        if result2[v] in list1:
            result2[v]=0
        elif result2[v] in list2:
            result2[v]=1
        else:
            print("-1")
    print(classification_report(y_test, result2))
    cm=confusion_matrix(y_test,result2)
    acc=metrics.accuracy_score(y_test,result2)
    print(str(acc))
    print(cm)

In [434]:
X_train = X_kpca[:len(df1)]
y_train = y[:len(df1)]
X_test = X_kpca[len(df1):]
y_test = y[len(df1):]
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
from imblearn.over_sampling import SMOTE
smote=SMOTE(sampling_strategy={0:12086})
X_train, y_train = smote.fit_resample(X_train, y_train)

In [435]:
CL_kmeans(X_train, X_test, y_train, y_test, 8)

              precision    recall  f1-score   support

           0       0.52      0.87      0.65      3178
           1       0.62      0.20      0.30      3178

    accuracy                           0.54      6356
   macro avg       0.57      0.54      0.48      6356
weighted avg       0.57      0.54      0.48      6356

0.538546255506608
[[2780  398]
 [2535  643]]


### Hyperparameter optimization of CL-k-means
Tune "k"

In [436]:
#Hyperparameter optimization by BO-GP
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from sklearn import metrics

space  = [Integer(2, 50, name='n_clusters')]
@use_named_args(space)
def objective(**params):
    km_cluster = MiniBatchKMeans(batch_size=100, **params)
    n=params['n_clusters']

    result = km_cluster.fit_predict(X_train)
    result2 = km_cluster.predict(X_val)

    count=0
    a=np.zeros(n)
    b=np.zeros(n)
    for v in range(0,n):
        for i in range(0,len(y_train)):
            if result[i]==v:
                if y_train[i]==1:
                    a[v]=a[v]+1
                else:
                    b[v]=b[v]+1
    list1=[]
    list2=[]
    for v in range(0,n):
        if a[v]<=b[v]:
            list1.append(v)
        else:
            list2.append(v)
    for v in range(0,len(y_val)):
        if result2[v] in list1:
            result2[v]=0
        elif result2[v] in list2:
            result2[v]=1
        else:
            print("-1")
    cm=metrics.accuracy_score(y_val,result2)
    print(str(n)+" "+str(cm))
    return (1-cm)
from skopt import gp_minimize
import time
t1=time.time()
res_gp = gp_minimize(objective, space, n_calls=20, random_state=0)
t2=time.time()
print(t2-t1)
print("Best score=%.4f" % (1-res_gp.fun))
print("""Best parameters: n_clusters=%d""" % (res_gp.x[0]))

30 0.9146314138471879
43 0.953170513107775
43 0.9511539323325118
43 0.9569796101277168
32 0.9271790275599373
20 0.941519157517365
16 0.9009634774815147
5 0.5119874523862873
15 0.904100380909702
25 0.9477929643737396
39 0.950929867801927
50 0.9594443199641497
47 0.9132870266636791
23 0.9446560609455523
36 0.9421913511091194
41 0.915751736500112
50 0.9513779968630965
50 0.9513779968630965
21 0.948465157965494
50 0.9549630293524535
5.4254150390625
Best score=0.9594
Best parameters: n_clusters=50


In [437]:
X_train = X_kpca[:len(df1)]
y_train = y[:len(df1)]
X_test = X_kpca[len(df1):]
y_test = y[len(df1):]
from imblearn.over_sampling import SMOTE
smote=SMOTE(sampling_strategy={0:12086})
X_train, y_train = smote.fit_resample(X_train, y_train)

In [438]:
CL_kmeans(X_train, X_test, y_train, y_test, res_gp.x[0])

              precision    recall  f1-score   support

           0       0.77      0.84      0.81      3178
           1       0.83      0.75      0.79      3178

    accuracy                           0.80      6356
   macro avg       0.80      0.80      0.80      6356
weighted avg       0.80      0.80      0.80      6356

0.7983008181246066
[[2682  496]
 [ 786 2392]]


### Apply the CL-k-means model with biased classifiers

95% of the code has been shared, and the remaining 5% is retained for future extension.  
Thank you for your interest and more details are in the paper.

In [439]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    recall_score
)
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.neighbors import NearestNeighbors
import numpy as np
from skopt import BayesSearchCV
from skopt.space import Integer
from imblearn.over_sampling import SMOTE

def Anomaly_IDS(X_train, X_test, y_train, y_test, n, b=100):
    km = MiniBatchKMeans(n_clusters=n, batch_size=b, random_state=0)
    train_labels = km.fit_predict(X_train)
    test_labels  = km.predict(X_test)

    counts_pos = np.zeros(n)
    counts_neg = np.zeros(n)
    for idx, c in enumerate(train_labels):
        if y_train[idx] == 1: counts_pos[c] += 1
        else:               counts_neg[c] += 1

    cluster_prob = {}
    normal_clusters = []
    attack_clusters = []
    for c in range(n):
        tot = counts_pos[c] + counts_neg[c]
        if counts_pos[c] > counts_neg[c]:
            attack_clusters.append(c)
            cluster_prob[c] = counts_pos[c] / tot if tot>0 else 0.0
        else:
            normal_clusters.append(c)
            cluster_prob[c] = counts_neg[c] / tot if tot>0 else 0.0

    y_km = np.array([1 if c in attack_clusters else 0 for c in test_labels])

    print("CL-k-means Performance:")
    print(classification_report(y_test, y_km))
    cm = confusion_matrix(y_test, y_km)
    tn, fp, fn, tp = cm.ravel()
    dr = tp/(tp+fn) if tp+fn>0 else 0
    far = fp/(fp+tn) if fp+tn>0 else 0
    print(f"  DR: {dr:.4f}, FAR: {far:.4f}\n  CM:\n{cm}\n")

    y_train_km = np.array([1 if c in attack_clusters else 0 for c in train_labels])
    fp_idx = np.where((y_train_km == 1) & (y_train == 0))[0]
    fn_idx = np.where((y_train_km == 0) & (y_train == 1))[0]

    X_fp = X_train[fp_idx]
    X_fn = X_train[fn_idx]
    X_norm = X_train[y_train == 0]
    X_adv  = X_train[y_train == 1]

    if len(X_fp) and len(X_adv):
        attack_samples_for_fp = X_adv[np.random.choice(len(X_adv), size=len(X_fp),replace=True)]
        Xp = np.concatenate([X_fp, attack_samples_for_fp])
        yp = np.concatenate([np.zeros(len(X_fp)), np.ones(len(X_fp))])
        yp = yp.astype(np.int64)
        y_counts = np.bincount(yp)
        print(y_counts)
        min_groups = np.min(y_counts)
        if min_groups >= 2:
          opt_rfp = BayesSearchCV(
              RandomForestClassifier(random_state=0),
              {
                  'n_estimators': Integer(10,200),
                  'max_depth':    Integer(3,50),
                  'min_samples_split': Integer(2,10)
              },
              n_iter=20,
              cv=StratifiedKFold(5, shuffle=True, random_state=0),
              scoring='f1',
              n_jobs=-1,
              random_state=0
          )
          opt_rfp.fit(Xp, yp)
          rfp = opt_rfp.best_estimator_
        else:
          rfp = RandomForestClassifier(random_state=0).fit(X_train, y_train)
    else:
        rfp = RandomForestClassifier(random_state=0).fit(X_train, y_train)

    if len(X_fn) and len(X_norm):
        normal_samples_for_fn = X_norm[np.random.choice(len(X_norm), size=len(X_fn),replace=True)]
        Xn = np.concatenate([X_fn, normal_samples_for_fn])
        yn = np.concatenate([np.zeros(len(X_fn)), np.ones(len(X_fn))])
        yn = yn.astype(np.int64)
        y_counts = np.bincount(yn)
        print(y_counts)
        min_groups = np.min(y_counts)

        if min_groups >= 2:
          opt_rfn = BayesSearchCV(
              RandomForestClassifier(random_state=0),
              {
                  'n_estimators': Integer(10,200),
                  'max_depth':    Integer(3,50),
                  'min_samples_split': Integer(2,10)
              },
              n_iter=20,
              cv=StratifiedKFold(5, shuffle=True, random_state=0),
              scoring='f1',
              n_jobs=-1,
              random_state=0
          )
          opt_rfn.fit(Xn, yn)
          rfn = opt_rfn.best_estimator_
        else:
          rfn = RandomForestClassifier(random_state=0).fit(X_train, y_train)  # fallback
    else:
        rfn = RandomForestClassifier(random_state=0).fit(X_train, y_train)  # fallback

    probs = np.array([cluster_prob.get(c,0.0) for c in test_labels])
    best_thr, best_rec = 0.5, 0.0
    for thr in np.linspace(0.5, 0.99, 50):
        y_tmp = y_km.copy()
        for i, p in enumerate(probs):
            if p < thr:
                if y_tmp[i] == 0:
                    y_tmp[i] = rfn.predict(X_test[i].reshape(1,-1))[0]
                else:
                    y_tmp[i] = rfp.predict(X_test[i].reshape(1,-1))[0]
        rec = recall_score(y_test, y_tmp)
        if rec > best_rec:
            best_rec, best_thr = rec, thr

    y_final = y_km.copy()
    for i, p in enumerate(probs):
        if p < best_thr:
            if y_final[i] == 0:
                y_final[i] = rfn.predict(X_test[i].reshape(1,-1))[0]
            else:
                y_final[i] = rfp.predict(X_test[i].reshape(1,-1))[0]


    print("MTH-IDS Performance:")
    print(classification_report(y_test, y_final))
    cm2 = confusion_matrix(y_test, y_final)

    tn, fp, fn, tp = cm2.ravel()
    dr2 = tp/(tp+fn) if tp+fn>0 else 0
    far2 = fp/(fp+tn) if fp+tn>0 else 0
    acc2 = accuracy_score(y_test, y_final)

    print(f"  Acc: {acc2:.4f}, DR: {dr2:.4f}, FAR: {far2:.4f}\n  CM:\n{cm2}")

    return acc2, dr2, far2, cm2

In [440]:
X_train = X_kpca[:len(df1)]
y_train = y[:len(df1)]
X_test = X_kpca[len(df1):]
y_test = y[len(df1):]
from imblearn.over_sampling import SMOTE
smote=SMOTE(sampling_strategy={0:12086})
X_train, y_train = smote.fit_resample(X_train, y_train)

In [441]:
one_acc, one_dr, one_far, one_f1 = Anomaly_IDS(X_train, X_test, y_train, y_test, res_gp.x[0])

CL-k-means Performance:
              precision    recall  f1-score   support

           0       0.68      0.88      0.77      3178
           1       0.83      0.58      0.68      3178

    accuracy                           0.73      6356
   macro avg       0.75      0.73      0.72      6356
weighted avg       0.75      0.73      0.72      6356

  DR: 0.5809, FAR: 0.1189
  CM:
[[2800  378]
 [1332 1846]]

[1446 1446]
[604 604]
MTH-IDS Performance:
              precision    recall  f1-score   support

           0       0.68      0.85      0.75      3178
           1       0.80      0.59      0.68      3178

    accuracy                           0.72      6356
   macro avg       0.74      0.72      0.72      6356
weighted avg       0.74      0.72      0.72      6356

  Acc: 0.7220, DR: 0.5916, FAR: 0.1476
  CM:
[[2709  469]
 [1298 1880]]
